In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv
/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv
/kaggle/input/dolly-v2-7b-8bit/config.json
/kaggle/input/dolly-v2-7b-8bit/tokenizer.json
/kaggle/input/dolly-v2-7b-8bit/tokenizer_config.json
/kaggle/input/dolly-v2-7b-8bit/pytorch_model.bin
/kaggle/input/dolly-v2-7b-8bit/special_tokens_map.json
/kaggle/input/dolly-v2-7b-8bit/generation_config.json
/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/falcon-7b-8bit/config.json
/kaggle/input/falcon-7b-8bit/configuration_falcon.py
/kaggle/input/falcon-7b-8bit/configuration_RW.py
/kaggle/input/falcon-7b-8bit/README.md
/kaggle/input/falcon-7b-8bit/tokenizer.json
/kaggle/input/falcon-7b-8bit/tokenizer_config.json
/kaggle/input/falcon-7b-8bit/modeling_falcon.py
/kaggle/input/falcon-7b-8bit/pytorch_model.bin.index.json
/kag

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
# Cloud AutoML
from google.cloud import automl_v1beta1 as automl
automl_client = automl.AutoMlClient()

# Cloud Translation
from google.cloud import translate_v2
translate_client = translate_v2.Client()

# Cloud Natural Language
from google.cloud import language_v1
client = language_v1.LanguageServiceClient()

# Cloud Video Intelligence
from google.cloud import videointelligence
video_client = videointelligence.VideoIntelligenceServiceClient()

# Cloud Vision
from google.cloud import vision
client = vision.ImageAnnotatorClient()

In [5]:
""" 
Credits
Takamichi Toda. 8-bit quantized LLM models: https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/424334#2343740
SFT: https://huggingface.co/docs/trl/main/en/sft_trainer
Perplexity: # https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking/notebook


"""

' \nCredits\nTakamichi Toda. 8-bit quantized LLM models: https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/424334#2343740\nSFT: https://huggingface.co/docs/trl/main/en/sft_trainer\nPerplexity: # https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking/notebook\n\n\n'

In [6]:
!pip install -q datasets transformers wandb sentencepiece accelerate

In [7]:
import pandas as pd

trainExtra_df = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv')
mainTrain_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
print(trainExtra_df.shape, mainTrain_df.shape)
mainTrain_df.head()

(6000, 7) (200, 8)


,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [8]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [9]:
iskaggle

'Interactive'

In [10]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"rajeshthevar","key":"ce00ae1ec01c54a33ad91cb57c09a25e"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!mkdir -p ./kaggle/input/einops ./kaggle/input/bitsandbytes ./kaggle/input/peft ./kaggle/input/accelerate ./kaggle/input/trl

In [12]:
!ls -d ./kaggle/input/*

./kaggle/input/accelerate    ./kaggle/input/einops  ./kaggle/input/trl
./kaggle/input/bitsandbytes  ./kaggle/input/peft


In [13]:
!pip download einops -d ./kaggle/input/einops
!pip download bitsandbytes -d ./kaggle/input/bitsandbytes
!pip download peft -d ./kaggle/input/peft
!pip download accelerate -d ./kaggle/input/accelerate
!pip download trl -d ./kaggle/input/trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
Saved ./kaggle/input/einops/einops-0.6.1-py3-none-any.whl
Successfully downloaded einops
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.0 MB/s eta 0:00:0000:0100:01
Saved ./kaggle/input/bitsandbytes/bitsandbytes-0.41.1-py3-none-any.whl
Successfully downloaded bitsandbytes
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 60.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
!ls -d ./kaggle/input/*

./kaggle/input/accelerate    ./kaggle/input/einops  ./kaggle/input/trl
./kaggle/input/bitsandbytes  ./kaggle/input/peft


In [15]:
import torch
from torch import nn

import gc
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [16]:
class CFG:
    INPUT_ROOT = "/kaggle/input/kaggle-llm-science-exam"
    MODEL_DIRS = [
        "/kaggle/input/dolly-v2-7b-8bit",
        "/kaggle/input/falcon-7b-8bit",
    ]

In [17]:
sub_df = pd.read_csv(f"{CFG.INPUT_ROOT}/sample_submission.csv")

In [18]:
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Submit")
    infer_df = pd.read_csv(f"{CFG.INPUT_ROOT}/test.csv")
    infer_df["answer"] = "A"
else:
    print("Test")
    infer_df = mainTrain_df
    
print(trainExtra_df.shape, mainTrain_df.shape)
infer_df.head()

Test
(6000, 7) (200, 8)


,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [19]:
class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
        self.reduce = reduce

    def forward(self, logits, labels):
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        #perplexity = torch.exp(perplexity)
        if self.reduce:
            perplexity = torch.mean(perplexity)
        return perplexity.cpu().item()
    
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

In [20]:
perplexity = Perplexity()
candidate = np.array(["A", "B", "C", "D", "E"])

In [28]:
def infer(model_name, infer_df):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
#         load_in_8bit=True,
        device_map="auto",
        trust_remote_code=True,
    )
    
    predicts, scores = [], []
    for _, row in tqdm(infer_df.iterrows(), total=len(infer_df)):
        inp = row["prompt"]
        cands = [f"Question: {inp}\nAnswer: {row[c]}" for c in candidate]

        inputs = tokenizer(
            cands,
            return_tensors="pt",
            add_special_tokens=False,
            padding=True, truncation=True
        ).to(f"cuda:{model.device.index}")
        with torch.no_grad():
            output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        output = output.logits
        labels = inputs["input_ids"]
        perp = [perplexity(output[i].unsqueeze(0), labels[i].unsqueeze(0)) for i in range(5)]
        pred = candidate[np.argsort(perp)]
        predicts.append(perp)

        s = MAP_at_3([pred], [row["answer"]])
        scores.append(s)

        torch.cuda.empty_cache()
        gc.collect()
        
    return predicts, scores

In [22]:
!ls -d ./kaggle/input/*

./kaggle/input/accelerate    ./kaggle/input/einops  ./kaggle/input/trl
./kaggle/input/bitsandbytes  ./kaggle/input/peft


In [23]:
!pip install einops --no-index --find-links=file:./kaggle/input/einops
!pip install bitsandbytes --no-index --find-links=file:./kaggle/input/bitsandbytes
!pip install accelerate --no-index --find-links=file:./kaggle/input/accelerate

Looking in links: file:///./kaggle/input/einops
Processing ./kaggle/input/einops/einops-0.6.1-py3-none-any.whl
Looking in links: file:///./kaggle/input/bitsandbytes
Processing ./kaggle/input/bitsandbytes/bitsandbytes-0.41.1-py3-none-any.whl
Looking in links: file:///./kaggle/input/accelerate


In [24]:
!pip install accelerate
!pip install bitsandbytes

In [29]:
predicts = []
import pdb

for model_name in CFG.MODEL_DIRS:
#     pdb.set_trace()
    _predicts, _scores = infer(model_name, infer_df)
    torch.cuda.empty_cache()
    gc.collect()
    print(model_name, np.mean(_scores))
    predicts.append(_predicts)

predicts = np.array(predicts)

Detected the presence of a `quantization_config` attribute in the model's configuration but you don't have the correct `bitsandbytes` version to support int8 serialization. Please install the latest version of `bitsandbytes` with  `pip install --upgrade bitsandbytes`.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3                                                                                             │
│    4 for model_name in CFG.MODEL_DIRS:                                                           │
│    5 #     pdb.set_trace()                                                                       │
│ ❱  6 │   _predicts, _scores = infer(model_name, infer_df)                                        │
│    7 │   torch.cuda.empty_cache()                                                                │
│    8 │   gc.collect()                                                                            │
│    9 │   print(model_name, np.mean(_scores))                                                     │
│                                                                                                  │
│ in infer:6                                                                                       │
│                                                                                                  │
│    3 │   if tokenizer.pad_token is None:                                                         │
│    4 │   │   tokenizer.pad_token = tokenizer.eos_token                                           │
│    5 │                                                                                           │
│ ❱  6 │   model = AutoModelForCausalLM.from_pretrained(                                           │
│    7 │   │   model_name,                                                                         │
│    8 │   │   torch_dtype=torch.bfloat16,                                                         │
│    9 #         load_in_8bit=True,                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:484 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 484 │   │   │   return model_class.from_pretrained(                                            │
│   485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2881 in from_pretrained   │
│                                                                                                  │
│   2878 │   │   │   │   mismatched_keys,                                                          │
│   2879 │   │   │   │   offload_index,                                                            │
│   2880 │   │   │   │   error_msgs,                                                               │
│ ❱ 2881 │   │   │   ) = cls._load_pretrained_model(                                               │
│   2882 │   │   │   │   model,                                                                    │
│   2883 │   │   │   │   state_dict,                         

In [1]:
if predicts.shape[0] == 1:
    predicts_avg = predicts[0]
else:
    predicts_avg = predicts.mean(0)

NameError: name 'predicts' is not defined

In [ ]:
sub_df["prediction"] = [" ".join(candidate[np.argsort(p)][:3]) for p in predicts_avg]

In [ ]:
sub_df.to_csv("submission.csv", index=None)

In [ ]:
pd.read_csv("submission.csv")

In [ ]:
!ls